<a href="https://colab.research.google.com/github/viniciuspxc/Projeto_02_LogiComp/blob/main/LogiComp_p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Declaração Python e PySat

In [2]:
!python -V

Python 3.7.13


In [3]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 18.0 MB/s 
     |████████████████████████████████| 3.4 MB 49.5 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=54606a8f6a1eed0d4e8850a886667f904aac1ff0faa8aef65863f59ed2964b8c
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious


In [4]:
from pysat.formula import IDPool
from pysat.formula import CNF
from pysat.solvers import Cadical
Atom = IDPool()

# Leitura de dados (Pandas)

In [139]:
import pandas as pd

In [140]:
dados= pd.read_csv("/content/drive/MyDrive/Trabalhos/Arquivos - Pacientes/column_bin_20a_30p.csv",sep=',')
dados.index += 1
dados

,PI <= 42.09,PI <= 48.12,PI <= 54.92,PI <= 63.52,PI <= 70.62,PI <= 80.61,PT <= 8.31,PT <= 12.36,PT <= 14.55,PT <= 17.44,...,PT <= 28.68,LA <= 32.59,LA <= 39.63,LA <= 46.33,LA <= 52.68,LA <= 61.27,LA <= 74.10,SS <= 29.51,SS <= 34.38,P
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,1
4,0,0,0,1,1,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,1
6,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
10,0,0,0,1,1,1,0,0,1,1,...,1,0,0,1,1,1,1,0,0,1


In [141]:
print("Pacientes: "+str(dados.shape[0]))
print("\nAtributos: "+str(list(dados)))
print("\nMatriz dos valores: "+str([tuple(x) for x in dados.values]))

Pacientes: 30

Atributos: ['PI <= 42.09', 'PI <= 48.12', 'PI <= 54.92', 'PI <= 63.52', 'PI <= 70.62', 'PI <= 80.61', 'PT <= 8.31', 'PT <= 12.36', 'PT <= 14.55', 'PT <= 17.44', 'PT <= 21.06', 'PT <= 28.68', 'LA <= 32.59', 'LA <= 39.63', 'LA <= 46.33', 'LA <= 52.68', 'LA <= 61.27', 'LA <= 74.10', 'SS <= 29.51', 'SS <= 34.38', 'P']

Matriz dos valores: [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1), (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1), (0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1), (0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1

# Quantidade de Regras

In [172]:
m = 4

# Restrições

## Funções Auxiliares

In [176]:
def lerdados(dados):
    pacientes=dados.shape[0]
    valor_atributos = [tuple(x) for x in dados.values]
    atributos = list(dados)
    return pacientes,atributos,valor_atributos

def watritbuto(a, i, t):
    return (Atom.id(a + '-' + str(i + 1) + '-' + t))

def wregra(i, j):
    return Atom.id('C-' + str(i + 1) + '-' + str(j + 1))

def pretty_formula_printer(formula):
  for clause in formula:
    for literal in clause:
      if literal > 0:
        print(Atom.obj(literal), ' ',  end = '')
      else:
        print('Not', Atom.obj(literal*-1), ' ',  end = '')
    print('')

## Restrição 1


Para cada atributo e cada regra, temos exatamente uma das três possibilidades: o atributo aparece positivamente na regra, o atributo aparece negativamente na regra ou o atributo não aparece na regra.

In [177]:
def r1(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    lista3 = []
    formula = []
    for i in range(m):
        for a in atributos:
            if a != 'P':
                lista=[]
                lista.append(watritbuto(a, i, 'p'))
                lista.append(watritbuto(a, i, 'n'))
                lista.append(watritbuto(a, i, 's'))
                lista2.append(lista)

                lista=[]
                lista.append(-watritbuto(a, i, 'p'))
                lista.append(-watritbuto(a, i, 'n'))
                lista2.append(lista)

                lista=[]
                lista.append(-watritbuto(a, i, 'p'))
                lista.append(-watritbuto(a, i, 's'))
                lista2.append(lista)

                lista=[]
                lista.append(-watritbuto(a, i, 'n'))
                lista.append(-watritbuto(a, i, 's'))
                lista2.append(lista)

    formula = lista2
    return formula

f1=r1(m,dados)
print(f1)
pretty_formula_printer(f1)

[[1, 2, 3], [-1, -2], [-1, -3], [-2, -3], [35, 36, 37], [-35, -36], [-35, -37], [-36, -37], [38, 39, 40], [-38, -39], [-38, -40], [-39, -40], [41, 42, 43], [-41, -42], [-41, -43], [-42, -43], [23, 24, 25], [-23, -24], [-23, -25], [-24, -25], [44, 45, 46], [-44, -45], [-44, -46], [-45, -46], [47, 48, 49], [-47, -48], [-47, -49], [-48, -49], [50, 51, 52], [-50, -51], [-50, -52], [-51, -52], [53, 54, 55], [-53, -54], [-53, -55], [-54, -55], [56, 57, 58], [-56, -57], [-56, -58], [-57, -58], [59, 60, 61], [-59, -60], [-59, -61], [-60, -61], [62, 63, 64], [-62, -63], [-62, -64], [-63, -64], [65, 66, 67], [-65, -66], [-65, -67], [-66, -67], [4, 5, 6], [-4, -5], [-4, -6], [-5, -6], [68, 69, 70], [-68, -69], [-68, -70], [-69, -70], [71, 72, 73], [-71, -72], [-71, -73], [-72, -73], [74, 75, 76], [-74, -75], [-74, -76], [-75, -76], [77, 78, 79], [-77, -78], [-77, -79], [-78, -79], [80, 81, 82], [-80, -81], [-80, -82], [-81, -82], [83, 84, 85], [-83, -84], [-83, -85], [-84, -85], [12, 13, 14], [-1

## Restrição 2

Cada regra deve ter algum atributo aparecendo nela.

In [178]:
def r2(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    for i in range(m):
        lista = []
        for a in atributos:
            if a != 'P':
                lista.append(-watritbuto(a, i, 's'))
        lista2.append(lista)
    formula = lista2
    return formula

f2=r2(m,dados)
print(f2)
pretty_formula_printer(f2)

[[-3, -37, -40, -43, -25, -46, -49, -52, -55, -58, -61, -64, -67, -6, -70, -73, -76, -79, -82, -85], [-14, -101, -104, -107, -31, -110, -113, -116, -119, -122, -125, -128, -131, -17, -134, -137, -140, -143, -146, -149], [-165, -168, -171, -174, -177, -180, -183, -186, -189, -192, -195, -198, -201, -204, -207, -210, -213, -216, -219, -222], [-240, -243, -246, -249, -252, -255, -258, -261, -264, -267, -270, -273, -276, -279, -282, -285, -288, -291, -294, -297]]
Not PI <= 42.09-1-s  Not PI <= 48.12-1-s  Not PI <= 54.92-1-s  Not PI <= 63.52-1-s  Not PI <= 70.62-1-s  Not PI <= 80.61-1-s  Not PT <= 8.31-1-s  Not PT <= 12.36-1-s  Not PT <= 14.55-1-s  Not PT <= 17.44-1-s  Not PT <= 21.06-1-s  Not PT <= 28.68-1-s  Not LA <= 32.59-1-s  Not LA <= 39.63-1-s  Not LA <= 46.33-1-s  Not LA <= 52.68-1-s  Not LA <= 61.27-1-s  Not LA <= 74.10-1-s  Not SS <= 29.51-1-s  Not SS <= 34.38-1-s  
Not PI <= 42.09-2-s  Not PI <= 48.12-2-s  Not PI <= 54.92-2-s  Not PI <= 63.52-2-s  Not PI <= 70.62-2-s  Not PI <= 8

## Restrição 3

 Para cada paciente sem patologia e cada regra, algum atributo do paciente não pode ser aplicado à regra.

In [179]:
def r3(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    for j in range(pacientes):
        if valor_atributos[j][atributos.index('P')] == 0:
            for i in range(m):
                lista = []
                for a in atributos:
                    if a != 'P':
                        if valor_atributos[j][atributos.index(str(a))] == 1: 
                            lista.append(watritbuto(a, i, 'n'))
                        else:
                            lista.append(watritbuto(a, i, 'p'))
                lista2.append(lista)
    formula = lista2
    return formula

f3=r3(m,dados)
print(f3)
pretty_formula_printer(f3)

[[1, 35, 39, 42, 24, 45, 47, 51, 54, 57, 60, 63, 65, 4, 69, 72, 75, 78, 80, 83], [12, 99, 103, 106, 30, 109, 111, 115, 118, 121, 124, 127, 129, 15, 133, 136, 139, 142, 144, 147], [163, 166, 170, 173, 176, 179, 181, 185, 188, 191, 194, 197, 199, 202, 206, 209, 212, 215, 217, 220], [238, 241, 245, 248, 251, 254, 256, 260, 263, 266, 269, 272, 274, 277, 281, 284, 287, 290, 292, 295], [1, 35, 38, 41, 23, 45, 47, 50, 53, 57, 60, 63, 65, 4, 68, 71, 75, 78, 80, 83], [12, 99, 102, 105, 29, 109, 111, 114, 117, 121, 124, 127, 129, 15, 132, 135, 139, 142, 144, 147], [163, 166, 169, 172, 175, 179, 181, 184, 187, 191, 194, 197, 199, 202, 205, 208, 212, 215, 217, 220], [238, 241, 244, 247, 250, 254, 256, 259, 262, 266, 269, 272, 274, 277, 280, 283, 287, 290, 292, 295], [1, 36, 39, 42, 24, 45, 47, 50, 54, 57, 60, 63, 66, 5, 69, 72, 75, 78, 81, 84], [12, 100, 103, 106, 30, 109, 111, 114, 118, 121, 124, 127, 130, 16, 133, 136, 139, 142, 145, 148], [163, 167, 170, 173, 176, 179, 181, 184, 188, 191, 194, 

## Restrição 4

Para cada paciente com patologia, cada regra e cada atributo, se o atributo do paciente não se aplicar ao da regra, então a regra não cobre esse paciente.

In [180]:
def r4(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    for j in range(pacientes):
        if valor_atributos[j][atributos.index('P')] == 1:
            for i in range(m):     
                for a in atributos:
                    lista = []
                    if a != 'P':
                        if valor_atributos[j][atributos.index(str(a))] == 1:
                            lista.append(-watritbuto(a, i, 'n'))
                            lista.append(-wregra(i, j))
                        else:
                            lista.append(-watritbuto(a, i, 'p'))
                            lista.append(-wregra(i, j))
                        lista2.append(lista)
    formula = lista2
    return formula

f4=r4(m,dados)
print(f4)
pretty_formula_printer(f4)

[[-1, -10], [-35, -10], [-38, -10], [-41, -10], [-23, -10], [-44, -10], [-47, -10], [-50, -10], [-53, -10], [-56, -10], [-59, -10], [-62, -10], [-65, -10], [-4, -10], [-68, -10], [-71, -10], [-75, -10], [-78, -10], [-80, -10], [-83, -10], [-12, -21], [-99, -21], [-102, -21], [-105, -21], [-29, -21], [-108, -21], [-111, -21], [-114, -21], [-117, -21], [-120, -21], [-123, -21], [-126, -21], [-129, -21], [-15, -21], [-132, -21], [-135, -21], [-139, -21], [-142, -21], [-144, -21], [-147, -21], [-163, -223], [-166, -223], [-169, -223], [-172, -223], [-175, -223], [-178, -223], [-181, -223], [-184, -223], [-187, -223], [-190, -223], [-193, -223], [-196, -223], [-199, -223], [-202, -223], [-205, -223], [-208, -223], [-212, -223], [-215, -223], [-217, -223], [-220, -223], [-238, -298], [-241, -298], [-244, -298], [-247, -298], [-250, -298], [-253, -298], [-256, -298], [-259, -298], [-262, -298], [-265, -298], [-268, -298], [-271, -298], [-274, -298], [-277, -298], [-280, -298], [-283, -298], [

## Restrição 5

Cada paciente com patologia deve ser coberto por alguma das regras.

In [181]:
def r5(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    for j in range(pacientes):
        lista = []
        if valor_atributos[j][atributos.index('P')] == 1:
          for i in range(m):
                lista.append(wregra(i, j))
          lista2.append(lista)
    formula = lista2
    return formula

f5=r5(m,dados)
print(f5)
pretty_formula_printer(f5)

[[10, 21, 223, 298], [11, 22, 224, 299], [86, 150, 225, 300], [87, 151, 226, 301], [88, 152, 227, 302], [89, 153, 228, 303], [90, 154, 229, 304], [91, 155, 230, 305], [92, 156, 231, 306], [93, 157, 232, 307], [94, 158, 233, 308], [95, 159, 234, 309], [96, 160, 235, 310], [97, 161, 236, 311], [98, 162, 237, 312]]
C-1-1  C-2-1  C-3-1  C-4-1  
C-1-2  C-2-2  C-3-2  C-4-2  
C-1-3  C-2-3  C-3-3  C-4-3  
C-1-4  C-2-4  C-3-4  C-4-4  
C-1-5  C-2-5  C-3-5  C-4-5  
C-1-6  C-2-6  C-3-6  C-4-6  
C-1-7  C-2-7  C-3-7  C-4-7  
C-1-8  C-2-8  C-3-8  C-4-8  
C-1-9  C-2-9  C-3-9  C-4-9  
C-1-10  C-2-10  C-3-10  C-4-10  
C-1-11  C-2-11  C-3-11  C-4-11  
C-1-12  C-2-12  C-3-12  C-4-12  
C-1-13  C-2-13  C-3-13  C-4-13  
C-1-14  C-2-14  C-3-14  C-4-14  
C-1-15  C-2-15  C-3-15  C-4-15  


# Execução

In [173]:
fr = r1(m,dados)+r2(m,dados)+r3(m,dados)+r4(m,dados)+r5(m,dados)
print(fr)

[[1, 2, 3], [-1, -2], [-1, -3], [-2, -3], [35, 36, 37], [-35, -36], [-35, -37], [-36, -37], [38, 39, 40], [-38, -39], [-38, -40], [-39, -40], [41, 42, 43], [-41, -42], [-41, -43], [-42, -43], [23, 24, 25], [-23, -24], [-23, -25], [-24, -25], [44, 45, 46], [-44, -45], [-44, -46], [-45, -46], [47, 48, 49], [-47, -48], [-47, -49], [-48, -49], [50, 51, 52], [-50, -51], [-50, -52], [-51, -52], [53, 54, 55], [-53, -54], [-53, -55], [-54, -55], [56, 57, 58], [-56, -57], [-56, -58], [-57, -58], [59, 60, 61], [-59, -60], [-59, -61], [-60, -61], [62, 63, 64], [-62, -63], [-62, -64], [-63, -64], [65, 66, 67], [-65, -66], [-65, -67], [-66, -67], [4, 5, 6], [-4, -5], [-4, -6], [-5, -6], [68, 69, 70], [-68, -69], [-68, -70], [-69, -70], [71, 72, 73], [-71, -72], [-71, -73], [-72, -73], [74, 75, 76], [-74, -75], [-74, -76], [-75, -76], [77, 78, 79], [-77, -78], [-77, -79], [-78, -79], [80, 81, 82], [-80, -81], [-80, -82], [-81, -82], [83, 84, 85], [-83, -84], [-83, -85], [-84, -85], [12, 13, 14], [-1

In [174]:
solution = Cadical()
solution.append_formula(fr)
if solution.solve():
    print("Valoração:\n")
    valoracao = solution.get_model()
    for atom in valoracao:
      if atom > 0:
        print(Atom.obj(atom),  end = ', ')
      else:
        print('Not(', Atom.obj(atom*-1), end = '), ')

else:
    print("A formula é insatisfativel, ou o valor de Regras(m) não foi suficiente")

Valoração:

Not( PI <= 42.09-1-p), Not( PI <= 42.09-1-n), PI <= 42.09-1-s, Not( LA <= 39.63-1-p), Not( LA <= 39.63-1-n), LA <= 39.63-1-s, GS <= 37.89-1-p, GS <= 37.89-1-n, GS <= 37.89-1-s, Not( C-1-1), Not( C-1-2), Not( PI <= 42.09-2-p), Not( PI <= 42.09-2-n), PI <= 42.09-2-s, Not( LA <= 39.63-2-p), Not( LA <= 39.63-2-n), LA <= 39.63-2-s, GS <= 37.89-2-p, GS <= 37.89-2-n, GS <= 37.89-2-s, Not( C-2-1), Not( C-2-2), Not( PI <= 70.62-1-p), Not( PI <= 70.62-1-n), PI <= 70.62-1-s, GS <= 57.55-1-p, GS <= 57.55-1-n, GS <= 57.55-1-s, Not( PI <= 70.62-2-p), Not( PI <= 70.62-2-n), PI <= 70.62-2-s, GS <= 57.55-2-p, GS <= 57.55-2-n, GS <= 57.55-2-s, Not( PI <= 48.12-1-p), Not( PI <= 48.12-1-n), PI <= 48.12-1-s, Not( PI <= 54.92-1-p), Not( PI <= 54.92-1-n), PI <= 54.92-1-s, Not( PI <= 63.52-1-p), Not( PI <= 63.52-1-n), PI <= 63.52-1-s, Not( PI <= 80.61-1-p), Not( PI <= 80.61-1-n), PI <= 80.61-1-s, PT <= 8.31-1-p, Not( PT <= 8.31-1-n), Not( PT <= 8.31-1-s), Not( PT <= 12.36-1-p), Not( PT <= 12.36-1-

In [175]:
def conj_regras(m, dados, valoracao):
    pacientes,atributos,valor_atributos = lerdados(dados)
    val = valoracao.copy()
    lista = []
    lista2 = []
    for i in range(m):
        lista=[]
        for a in atributos:
            if a != 'P':
                if (-watritbuto(a, i, 's') in valoracao):
                    if (watritbuto(a, i, 'n') in valoracao):
                        a2 = a.replace("<=", ">")
                        lista.append(a2)
                    else:
                        lista.append(a)
        lista2.append(str(lista) + ' \u21E8 P')
    return lista2

if solution.solve():
   regras = conj_regras(m,dados,valoracao)
   print("Conjunto de Regras:\n")
   for lin in regras:
     print(lin)

Conjunto de Regras:

['PT <= 8.31', 'PT <= 14.55', 'LA > 52.68'] ⇨ P
['PI > 54.92', 'PT <= 14.55'] ⇨ P
['LA > 61.27'] ⇨ P
['PT > 17.44'] ⇨ P
